# Abandono de clientes en una compañía telefónica

## Caso de Uso  

El ejemplo que vamos a utilizar proviene de un dataset de uso libre disponible en [Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn).

El conjunto de datos incluye información sobre:
* __Clientes que se fueron en el último mes__: la columna se llama _Churn_  
* __Servicios para los que se ha suscrito cada cliente__: teléfono, varias líneas, Internet, seguridad en línea, respaldo en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas  
* __Información de la cuenta del cliente__: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales  
* __Información demográfica sobre los clientes__: sexo, rango de edad y si tienen socios y dependientes


***

## Configuración de la conexión con Vantage

In [ ]:
%connect Transcend-Production

In [ ]:
DATABASE ADLSLSAMER_MS_AZ;

## Acceso y Exploración de los Datos

In [ ]:
SELECT TOP 10 * FROM TelcoCustomerChurn

### Exploración de Datos  
 
Vamos a utilizar la función _DataExplorer_ de __Vantage Analytic Library__, Esta función genera varias tablas donde se almacenan los resultados del análisis.   
Por eso, primero vamos a eliminar las tablas que contienen el resultado del análisis exploratorio previo para poder recrearlas.

In [ ]:
DROP TABLE TwmExploreValues;

In [ ]:
DROP TABLE TwmExploreStatistics;

In [ ]:
DROP TABLE TwmExploreFrequency;

In [ ]:
DROP TABLE TwmExploreHistogram;

In [ ]:
call TRNG_XSP.td_analyze('DataExplorer',
                      'database=ADLSLSAMER_DS_GENERAL;
                       tablename=TelcoCustomerChurn;
                       outputdatabase=ADLSLSAMER_DS_GENERAL;');

Ahora vamos a revisar los resultados de nuestro Análisis Exploratorio.  
  
#### 1. Exploratorio de Valores  
  
Nos muestra los descriptivos generales de las variables en la tabla:  
* Tipo de variable
* Cantidad de registros
* Cantidad de nulos
* etc.

In [ ]:
SELECT * FROM TwmExploreValues ORDER BY xtype, xcol;

#### 2. Descriptivo de variables numéricas  
  
Nos muestra los descriptivos básicos de las variables numéricas en nuestra tabla de datos.  
* Mínimo
* Máximo
* Promedio
* Desvío estándar

In [ ]:
SELECT * FROM TwmExploreStatistics ORDER BY xcol;

#### 3. Descriptivos de variables de texto  
  
Nos muestra los descriptivos básicos de las variables de texto en nuestra tabla de datos.  
* Valores encontrados
* Cantidad de registros para cada valor
* Porcentaje de registros para cada valor

In [ ]:
SELECT * FROM TwmExploreFrequency ORDER BY xcol, xval;

##### Ejemplo de un gráfico de sectores

In [ ]:
SELECT xval as Codigo,xcnt as cantidad FROM TwmExploreFrequency WHERE xcol = 'gender';

In [ ]:
%chart Codigo, cantidad, title=Distribución de Codigo de Area, labelx=Metodo, labely=Count, width=800, height=450

#### 4. Descriptivos de variables numéricas discretizadas (bins)  
  
Nos muestra los descriptivos resultantes de discretizar las variables numéricas de nuestra tabla de datos.
* Número de segmento
* Valor inicial del segmento
* Valor final del segmento
* Cantidad de casos dentro del segmento
* Porcentaje que representan esos casos

In [ ]:
SELECT * FROM TwmExploreHistogram ORDER BY xcol, xbin;

##### Gráfico de bins

In [ ]:
SELECT xbin as bin,xcnt as cantidad FROM TwmExploreHistogram WHERE xcol = 'TotalCharges';

In [ ]:
%chart bin, cantidad, title=Distribución de Llamadas en el Día, labelx=Bins, labely=Count, width=800, height=450

## Transformación de datos  
 
En general es necesario realizar transformaciones sobre los datos para adecuarlos al tipo de análisis que planificamos realizar. 
  
La idea es crear un modelo de tipo __Regresión Logística__ que tiene exigencias específicas en cuanto al tipo de variables a utilizar
  
Por eso modificaremos las variables para que sean adecuadas para el modelo  
  
Para eso presentamos a modo de ejemplo dos tipos de transformaciones comunes:  
  
* Transformaciones de variables de cadena en un conjunto de marcas o flags 0-1  
* Estandarización de las variables numéricas para que tengan promedio 0 y desvío estándar 1

### 1. Transformación de variables de cadena en marcas.  
    
Utilizaremos como ejemplo las siguientes variables de cadena:
* gender,  
* Partner,  
* Dependents,  
* PhoneService,  
* MultipleLines,  
* InternetService,  
* OnlineSecurity,  
* OnlineBackup,  
* DeviceProtection,  
* TechSupport,  
* StreamingTV,  
* StreamingMovies,  
* Contract,  
* PaperlessBilling,  
* PaymentMethod,  
* Churn  
    
Y para cada una de ellas vamos a utilizar la función _OneHotEncoder_ para generar el conjunto de marcas.

In [ ]:
DROP TABLE tch_varflags;

In [ ]:
call TRNG_XSP.td_analyze('vartran',
    'database=ADLSLSAMER_MS_AZ;
    tablename=TelcoCustomerChurn;
    outputstyle=table;
    outputdatabase=ADLSLSAMER_MS_AZ;
    outputtablename=tch_varflags;
    keycolumns=customerid;
    index=customerid;
    designcode=
    {designstyle(dummycode),designvalues(Female/FLAG_Gender),columns(Gender)}
    {designstyle(dummycode),designvalues(Yes/FLAG_Partner),columns(Partner)}
    {designstyle(dummycode),designvalues(Yes/FLAG_Dependents),columns(Dependents)}
    {designstyle(dummycode),designvalues(Yes/FLAG_Phone),columns(PhoneService)}
    {designstyle(dummycode),designvalues(Yes/FLAG_ml_yes,No phone service/FLAG_ml_nphs),columns(MultipleLines)}
    {designstyle(dummycode),designvalues(Fiber Optic/FLAG_is_fibopt,DSL/FLAG_is_dsl),columns(InternetService)}
    {designstyle(dummycode),designvalues(Yes/FLAG_os_yes,No/FLAG_os_no),columns(OnlineSecurity)}
    {designstyle(dummycode),designvalues(Yes/FLAG_ob_yes,No/FLAG_ob_no),columns(OnlineBackup)}
    {designstyle(dummycode),designvalues(Yes/FLAG_dp_yes,No/FLAG_dp_no),columns(DeviceProtection)}
    {designstyle(dummycode),designvalues(Yes/FLAG_ts_yes,No/FLAG_ts_no),columns(TechSupport)}
    {designstyle(dummycode),designvalues(Yes/FLAG_stv_yes,No/FLAG_stv_no),columns(StreamingTV)}
    {designstyle(dummycode),designvalues(Yes/FLAG_stm_yes,No/FLAG_stm_no),columns(StreamingMovies)}
    {designstyle(dummycode),designvalues(One Year/FLAG_ctr_oneyear,Two year/FLAG_ctr_twoyear),columns(Contract)}
    {designstyle(dummycode),designvalues(Yes/FLAG_plbill),columns(PaperlessBilling)}
    {designstyle(dummycode),designvalues(Mailed check/FLAG_pm_check,Electronic check/FLAG_pm_echeck),columns(PaymentMethod)}
    {designstyle(dummycode),designvalues(Yes/FLAG_churn),columns(Churn)};');

In [ ]:
SELECT TOP 10 * FROM tch_varflags

### 2. Valor Z para variables numéricas.  
    
Utilizaremos como ejemplo las siguientes variables numéricas:
* MonthlyCharges,  
* TotalCharges,  
* Tenure
    
Y para cada una de ellas vamos a utilizar la función _ZScore_ para generar la transformación.    

In [ ]:
DROP TABLE tch_zvar;

In [ ]:
call TRNG_XSP.td_analyze('vartran',
    'database=ADLSLSAMER_MS_AZ;
    tablename=TelcoCustomerChurn;
    outputstyle=table;
    outputdatabase=ADLSLSAMER_MS_AZ;
    outputtablename=tch_zvar;
    keycolumns=customerid;
    index=customerid;
    zscore=columns(MonthlyCharges/zMonthlyCharges,TotalCharges/zTotalCharges,Tenure/zTenure);
;');

In [ ]:
SELECT TOP 10 * FROM tch_zvar;

### 3. Unimos los datos transformados y generamos marcas para las muestras de Entrenamiento, Validación y Scoring  
   
Sobre nuestro __Dataset final__ generaremos una marca al azar __para crear y evaluar__ el modelo y reservaremos una muestra para __aplicar__ posteriormente el modelo en producción  

In [ ]:
DROP TABLE TelcoCustomerChurn_dataset_sql;  

In [ ]:
CREATE TABLE TelcoCustomerChurn_dataset_sql AS
(
SELECT a.customerid,a.FLAG_churn,a.FLAG_Gender,a.FLAG_Partner,a.FLAG_Dependents,a.FLAG_Phone,a.FLAG_ml_yes,
    a.FLAG_ml_nphs,a.FLAG_is_fibopt,a.FLAG_is_dsl,a.FLAG_os_yes,a.FLAG_os_no,a.FLAG_ob_yes,a.FLAG_ob_no,
    a.FLAG_dp_yes,a.FLAG_dp_no,a.FLAG_ts_yes,a.FLAG_ts_no,a.FLAG_stv_yes,a.FLAG_stv_no,a.FLAG_stm_yes,
    a.FLAG_stm_no,a.FLAG_ctr_oneyear,a.FLAG_ctr_twoyear,a.FLAG_plbill,a.FLAG_pm_check,
    a.FLAG_pm_echeck,b.zMonthlyCharges,b.zTotalCharges,b.zTenure,
        SAMPLEID as sid
FROM tch_varflags a
LEFT JOIN tch_zvar b
ON a.customerid=b.customerid
SAMPLE RANDOMIZED ALLOCATION 0.3, 0.3, 0.4
) WITH DATA
PRIMARY INDEX (customerid);

In [ ]:
SELECT TOP 10 * FROM TelcoCustomerChurn_dataset_sql;

### 4. Separación de las muestras de entrenamiento, validación y scoring

In [ ]:
DROP TABLE tch_train;

In [ ]:
CREATE MULTISET TABLE tch_train AS (
SELECT * FROM TelcoCustomerChurn_dataset_sql WHERE SID = 1
) WITH DATA
PRIMARY INDEX (customerid);

In [ ]:
DROP TABLE tch_test;

In [ ]:
CREATE MULTISET TABLE tch_test AS (
SELECT * FROM TelcoCustomerChurn_dataset_sql WHERE SID = 2
) WITH DATA
PRIMARY INDEX (customerid);

In [ ]:
DROP TABLE tch_scoring;

In [ ]:
CREATE MULTISET TABLE tch_scoring AS (
SELECT * FROM TelcoCustomerChurn_dataset_sql WHERE SID = 3
) WITH DATA
PRIMARY INDEX (customerid);

## Creación y Validación de un modelo de Regresión Logística
  
El algoritmo de Regresión Logística contiene distintos parámetros para su ejecución.  
Aquí vamos a mostrar una de las configuraciones posibles, puede encontrar más información [Teradata Documentation/Vantage Analytics Library User Guide/Logistic Regression](https://docs.teradata.com/r/DqJoLxuCtoR947URt1aH0w/a9t1K0AD6cMnV1zPRgflVg).  

Primero eliminamos las tablas que contienen el resultado de la regresión logística para poder recrearlas.

In [ ]:
DROP TABLE tch_logmodel;

In [ ]:
DROP TABLE tch_logmodel_rpt;

In [ ]:
DROP TABLE tch_logmodel_txt;

Ahora crearemos el modelo

In [ ]:
call TRNG_XSP.td_analyze('logistic','
database=ADLSLSAMER_MS_AZ;
tablename=tch_train;
columns= all;
columnstoexclude=customerid,sid;
dependent=FLAG_Churn;
response=1;
stepwise=true;
statstable=true;
successtable=true;
thresholdtable=true;
lifttable=true;
outputdatabase=ADLSLSAMER_MS_AZ;
outputtablename=tch_logmodel;');

#### Revisamos el Modelo

In [ ]:
SELECT * FROM tch_logmodel;

In [ ]:
SELECT * FROM tch_logmodel_rpt;

In [ ]:
SELECT * FROM tch_logmodel_txt;

***
    
### Evaluamos el modelo de Regresión Logística.

In [ ]:
DROP TABLE tch_logmodelval;

In [ ]:
DROP TABLE tch_logmodelval_txt;

In [ ]:
call
TRNG_XSP.td_analyze('logisticscore','
database=ADLSLSAMER_MS_AZ;
tablename=tch_test;
modeldatabase=ADLSLSAMER_MS_AZ;
modeltablename=tch_logmodel;
outputdatabase=ADLSLSAMER_MS_AZ;
outputtablename=tch_logmodelval;
estimate=Estimate;
probability=Probability;
retain=FLAG_churn;
samplescoresize=10;
scoringmethod=scoreandevaluate;
lifttable=true;');

In [ ]:
SELECT FLAG_churn, Estimate, COUNT(1) AS n FROM tch_logmodelval GROUP BY 1,2;

*** 
    
### Aplicamos el modelo de Regresión Logística a un nuevo Dataset.

In [ ]:
DROP TABLE tch_score;

In [ ]:
call
TRNG_XSP.td_analyze('logisticscore','
database=ADLSLSAMER_MS_AZ;
tablename=tch_scoring;
modeldatabase=ADLSLSAMER_MS_AZ;
modeltablename=tch_logmodel;
outputdatabase=ADLSLSAMER_MS_AZ;
outputtablename=tch_score;
probability=Probability;
scoringmethod=score;');

In [ ]:
SELECT TOP 10 * FROM tch_score;

In [ ]:
%disconnect Transcend-Production

©2022 Teradata. All Rights Reserved.